In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("s8a-dataframes-sql").getOrCreate()

# Lectura de CSV con el ; como separador de columnas y con encabezado
df = spark.read.option("sep",";").option("header", "true").option("inferSchema", "true").csv("pdi_sales_small.csv")
    
df.printSchema()

22/03/29 11:49:37 WARN Utils: Your hostname, MacBook-Air-de-Aitor.local resolves to a loopback address: 127.0.0.1; using 192.168.1.203 instead (on interface en0)
22/03/29 11:49:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/29 11:49:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- ProductID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Country: string (nullable = true)



In [2]:
spark

In [3]:
df.show(4)

+---------+---------+---------------+-----+-------+-------+
|ProductID|     Date|            Zip|Units|Revenue|Country|
+---------+---------+---------------+-----+-------+-------+
|      725|1/15/1999|41540          |    1|  115.5|Germany|
|      787| 6/6/2002|41540          |    1|  314.9|Germany|
|      788| 6/6/2002|41540          |    1|  314.9|Germany|
|      940|1/15/1999|22587          |    1|  687.7|Germany|
+---------+---------+---------------+-----+-------+-------+
only showing top 4 rows



In [4]:
from pyspark.sql.functions import col, trim
df = df.withColumn("Country", trim(col("Country"))).withColumn("Zip", trim(col("Zip")))

In [5]:
from pyspark.sql.functions import to_date
df = df.withColumn("Date", to_date(df.Date, "M/d/yyy"))

In [6]:
from pyspark.sql.functions import year
df.select(year("Date")).distinct().show()

+----------+
|year(Date)|
+----------+
|      2003|
|      2004|
|      2001|
|      2000|
|      1999|
|      2002|
+----------+



In [7]:
df.createOrReplaceTempView("ventas")

In [8]:
spark.sql("select count(Country) from ventas").show()

+--------------+
|count(Country)|
+--------------+
|        120239|
+--------------+



## Contando

In [9]:
from pyspark.sql.functions import count
df.select(count("Country")).show()

+--------------+
|count(Country)|
+--------------+
|        120239|
+--------------+



In [10]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("Country"), countDistinct("Zip")).show()

+-----------------------+-------------------+
|count(DISTINCT Country)|count(DISTINCT Zip)|
+-----------------------+-------------------+
|                      4|               2585|
+-----------------------+-------------------+



In [11]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("Country"), approx_count_distinct("Zip")).show()

22/03/29 11:50:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------+--------------------------+
|approx_count_distinct(Country)|approx_count_distinct(Zip)|
+------------------------------+--------------------------+
|                             4|                      2575|
+------------------------------+--------------------------+



## Calculando

In [12]:
from pyspark.sql.functions import min, max
df.select(min("Units"), max("Units")).show()

+----------+----------+
|min(Units)|max(Units)|
+----------+----------+
|         1|        77|
+----------+----------+



In [13]:
from pyspark.sql.functions import sum
df.select(sum("Units"), sum("Revenue")).show()

+----------+--------------------+
|sum(Units)|        sum(Revenue)|
+----------+--------------------+
|    125728|5.0107274999986745E7|
+----------+--------------------+



In [14]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Units"), sumDistinct("Revenue")).show()

+-------------------+---------------------+
|sum(DISTINCT Units)|sum(DISTINCT Revenue)|
+-------------------+---------------------+
|                308|            1189127.1|
+-------------------+---------------------+



In [15]:
from pyspark.sql.functions import sum, count, avg
df.select(avg("Revenue"), sum("Revenue")/count("Revenue")).show()

+-----------------+-------------------------------+
|     avg(Revenue)|(sum(Revenue) / count(Revenue))|
+-----------------+-------------------------------+
|416.7306364822291|              416.7306364822291|
+-----------------+-------------------------------+



## Agrupando

In [16]:
from pyspark.sql.functions import sum
df.groupBy("Country").sum("Revenue").show()

+-------+--------------------+
|Country|        sum(Revenue)|
+-------+--------------------+
|Germany|1.4982119999999512E7|
| France|1.2087942100000832E7|
| Mexico| 1.139459870000116E7|
| Canada|1.1642614200001905E7|
+-------+--------------------+



In [17]:
from pyspark.sql.functions import sum
df.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|Germany|30059|
| France|30060|
| Mexico|30060|
| Canada|30060|
+-------+-----+



In [18]:
df.groupBy("Country").agg(sum("Revenue"), count("Revenue")).show()

+-------+--------------------+--------------+
|Country|        sum(Revenue)|count(Revenue)|
+-------+--------------------+--------------+
|Germany|1.4982119999999512E7|         30059|
| France|1.2087942100000832E7|         30060|
| Mexico| 1.139459870000116E7|         30060|
| Canada|1.1642614200001905E7|         30060|
+-------+--------------------+--------------+



In [19]:
df.groupBy("Country").agg({"Zip":"count", "Revenue":"avg"}).show()

+-------+----------+------------------+
|Country|count(Zip)|      avg(Revenue)|
+-------+----------+------------------+
|Germany|     30059| 498.4237665923521|
| France|     30060| 402.1271490352905|
| Mexico|     30060| 379.0618330007039|
| Canada|     30060|387.31251497012323|
+-------+----------+------------------+



In [20]:
from pyspark.sql.functions import collect_list, collect_set
df.where("Units > 5").groupBy("Country").agg(collect_list("Zip"), collect_set("Zip")).show()

+-------+--------------------+--------------------+
|Country|   collect_list(Zip)|    collect_set(Zip)|
+-------+--------------------+--------------------+
|Germany|[22397, 22111, 40...|[22111, 12589, 22...|
| France|[75213 CEDEX 16, ...|[06082 CEDEX 1, 0...|
| Mexico|[7100, 7810, 9739...|[9739, 10300, 781...|
| Canada|[T2X, V6G, V6G, T6V]|     [V6G, T2X, T6V]|
+-------+--------------------+--------------------+



In [21]:
from pyspark.sql.functions import year
df.select(year("Date")).distinct().show()

+----------+
|year(Date)|
+----------+
|      2003|
|      2004|
|      2001|
|      2000|
|      1999|
|      2002|
+----------+



In [22]:
df.groupBy(year("Date")).pivot("Country").sum("Revenue").show()

+----------+------------------+------------------+------------------+------------------+
|year(Date)|            Canada|            France|           Germany|            Mexico|
+----------+------------------+------------------+------------------+------------------+
|      2003| 2360085.999999947|1105230.9000000046|1407120.0000000007|         1049457.5|
|      2004| 1539140.499999946|              null|              null|              null|
|      2001| 2193437.799999908|              null|              null|233419.20000000004|
|      2000|1806678.3999999042|1108846.8999999764| 4510606.799999941| 4240448.399999928|
|      1999|1382756.6999999764| 7594921.200000435| 5928459.100000297|3419368.2000001906|
|      2002|2360514.7999998857| 2278943.099999957| 3135934.099999964|2451905.3999999263|
+----------+------------------+------------------+------------------+------------------+



In [23]:
df.groupBy(year("Date")).pivot("Country").agg(sum("Revenue").alias("total"), sum("Units").alias("cantidad")).show()

+----------+------------------+---------------+------------------+---------------+------------------+----------------+------------------+---------------+
|year(Date)|      Canada_total|Canada_cantidad|      France_total|France_cantidad|     Germany_total|Germany_cantidad|      Mexico_total|Mexico_cantidad|
+----------+------------------+---------------+------------------+---------------+------------------+----------------+------------------+---------------+
|      2003| 2360085.999999947|           6375|1105230.9000000046|           2794|1407120.0000000007|            3099|         1049457.5|           2510|
|      2004| 1539140.499999946|           3636|              null|           null|              null|            null|              null|           null|
|      2001| 2193437.799999908|           5976|              null|           null|              null|            null|233419.20000000004|            583|
|      2000|1806678.3999999042|           5049|1108846.8999999764|          